In [ ]:
import pandas as pd
dataset = pd.read_csv('Overall Movie.csv')
dataset = dataset.loc[dataset['Reviews'] != '[]']
# reset index of dataset
dataset = dataset.reset_index(drop=True)
# save as csv
dataset.to_csv('Overall Movie (removed).csv', index=False)
# Convert all reviews to lowercase
dataset['Reviews'] = dataset['Reviews'].apply(lambda x: x.lower())

# Concat description and reviews and Stars to string

dataset['Reviews'] = dataset['Reviews'].astype(str)
dataset['Reviews'] = dataset['Description'] + dataset['Reviews']

# Remove "[" and "]"
dataset['Reviews'] = dataset['Reviews'].apply(lambda x: x.replace("['",''))
dataset['Reviews'] = dataset['Reviews'].apply(lambda x: x.replace("']",''))

dataset



In [3]:
import pandas as pd
dataset = pd.read_csv('Overall Movie (genre).csv')

# reset index of dataset
dataset = dataset.reset_index(drop=True)

# concat all columns
dataset['total'] = dataset['Movie Name'] + " " + dataset['Year of Release'].astype(str) + " " + dataset['Watch Time'].astype(str) + " " + dataset['Movie Rating'].astype(str) + " " + dataset['Director'] + " " + dataset['Genre'] + " " + dataset['Cast'] + " " + dataset['Description']

# remove all special characters
dataset['total'] = dataset['total'].astype(str).apply(lambda x: x.replace("[^a-zA-Z#]", " "))

dataset

,Unnamed: 0,Movie Name,Year of Release,Watch Time,Movie Rating,Director,Description,Cast,Genre,total
0,2.0,Journey,2023,120,8.4,['Cristopher Rodriguez'],Roberto is a young man who is trying to find h...,"['Meaghan Martin', 'Rafael Cebrián', 'Gustavo ...",Drama,Journey 2023 120 8.4 ['Cristopher Rodriguez'] ...
1,4.0,Peter Pan & Wendy,2023,106,4.1,['David Lowery'],"Follow the adventures of Peter Pan, a boy who ...","['Alexander Molony', 'Ever Anderson', 'Jude La...","Action,Adventure,Comedy",Peter Pan & Wendy 2023 106 4.1 ['David Lowery'...
2,5.0,Sid Is Dead,2023,90,5.6,['Eli Gonda'],The story of a socially invisible high school ...,"['Mary Stuart Masterson', 'Carla Gallo', 'Gene...","Comedy,Drama","Sid Is Dead 2023 90 5.6 ['Eli Gonda'] Comedy,D..."
3,6.0,Snag: Chapter One,2023,83,4.1,['Ben Milliken'],An Australian lone wolf's quiet existence is s...,"['Ben Milliken', 'Sofía Castro', 'Jonny Beauch...",NaN,nan
4,7.0,Chang kong zhi wang,2023,128,6.6,['Xiaoshi Liu'],A special operations team headed by elite pilo...,"['Yibo Wang', 'Jun Hu', 'Yosh Yu', 'Dongyu Zhou']","Action,Drama",Chang kong zhi wang 2023 128 6.6 ['Xiaoshi Liu...
...,...,...,...,...,...,...,...,...,...,...
47110,NaN,Shadowzone,1990,88,5.1,['J.S. Cardone'],Scientists experiment with Extreme Dream Sleep...,"['Louise Fletcher', 'David Beecroft', 'James H...","Horror,Sci-Fi",Shadowzone 1990 88 5.1 ['J.S. Cardone'] Horror...
47111,NaN,Why Me?,1990,87,5.8,['Gene Quintano'],A jewel thief steals a sacred ruby which sets ...,"['Christopher Lambert', 'Kim Greist', 'Christo...","Action,Adventure,Comedy","Why Me? 1990 87 5.8 ['Gene Quintano'] Action,A..."
47112,NaN,On lok jin cheung,1990,96,6.3,['Eric Tsang'],A pleasant trip to the Phillipines is turned i...,"['Eric Tsang', 'Irene Wan', 'Kwong-Leung Wong'...","Action,Crime,Drama",On lok jin cheung 1990 96 6.3 ['Eric Tsang'] A...
47113,NaN,Love or Money,1990,90,4.5,['Todd Hallowell'],Two young owners of a firm are about to make a...,"['Tim Daly', 'Michael Garin', 'Haviland Morris...",Comedy,Love or Money 1990 90 4.5 ['Todd Hallowell'] C...


In [5]:
from sentence_transformers import SentenceTransformer, util
import torch

model = SentenceTransformer('all-mpnet-base-v2')
model.max_seq_length = 200

embeddings2 = model.encode(dataset['total'].tolist(), convert_to_tensor=True)
# save tensors as a file
torch.save(embeddings2, 'Overall Movie (genre).pt')

In [7]:
model = SentenceTransformer('all-mpnet-base-v2')
model.max_seq_length = 200

tensors = torch.load('Overall Movie (genre).pt')
movie_1 = "The Dark Knight"
movie_1_rating = 2
movie_2 = "Spongebob"
movie_2_rating = 10
movie_1_tensors = model.encode(movie_1, convert_to_tensor=True)
movie_2_tensors = model.encode(movie_2, convert_to_tensor=True)

# find weighted cosine similarities using the movie ratings
cosine_scores = util.pytorch_cos_sim(movie_1_tensors, tensors) * movie_1_rating + util.pytorch_cos_sim(movie_2_tensors, tensors) * movie_2_rating
top_results = torch.topk(cosine_scores, k=70)
top_indices = top_results[1][0]
top_scores = top_results[0][0]


# print top 20 movies and scores
for i in range(20):
    print(dataset['Movie Name'][top_indices[i].item()], top_scores[i].item())


The SpongeBob Movie: Sponge Out of Water 6.580746650695801
The SpongeBob Movie: Sponge on the Run 6.566628932952881
The SpongeBob SquarePants Movie Rehydrated 6.511837959289551
The SpongeBob SquarePants Movie 6.511575698852539
A Day With SpongeBob SquarePants 6.2143874168396
Under the Sea 4.653863430023193
Scoob! 4.598236560821533
Echoboom 4.569362640380859
Squid Man 4.568920612335205
The Squid and the Whale 4.5303120613098145
Jay and Silent Bob's Super Groovy Cartoon Movie 4.498553276062012
BoBoiBoy Movie 2 4.497082710266113
Go Fish 4.491065502166748
Trolls 4.4353156089782715
Finding Dory 4.434197902679443
Ralph Breaks the Internet 4.430570602416992
Bob Mazooka: Fun Time Action Hour for Kids 4.419001579284668
Uh Oh! 4.373802185058594
Mr. Gibb 4.348344802856445
Captain Underpants: The First Epic Movie 4.34480094909668
